In [ ]:
import pyspark
import json

In [ ]:
conf = pyspark.SparkConf()
conf.setAppName("Twitter page rank")
conf.setMaster("local[*]")

In [ ]:
sc = pyspark.SparkContext(conf = conf)

In [ ]:
twitterLines = sc.textFile("twitter_closed.jsonl")

In [ ]:
followers = twitterLines.map(lambda line: json.loads(line))

In [ ]:
links = followers\
    .keyBy(lambda d: d["user_id"])\
    .mapValues(lambda d: d["following_id"])

In [ ]:
N = links.count()
d = 0.85

In [ ]:
ranks = links\
    .keyBy(lambda usuario: usuario[0])\
    .map(lambda usuario: (usuario[0], 1 / N))

In [ ]:
iterations = 10

In [ ]:
def merge_resources(row):
    
    resultados = list()
    usuario = row[0]
    seguidos = row[1][0]
    page_rank_usuario = row[1][1] or 1 / N
    seguidos_usuario = len(seguidos)

    for seguido in seguidos:
        resultados.append((seguido, page_rank_usuario / seguidos_usuario))
    resultados.append((usuario, 0))
    
    return resultados

In [ ]:
rranks = dict(ranks.collect())
ffollowers = dict(links.collect())

In [ ]:
for i in range(iterations):
    
    contributions = links.fullOuterJoin(ranks)\
        .flatMap(merge_resources)
    
    ranks = contributions\
        .reduceByKey(lambda a, b: a + b)\
        .mapValues(lambda v: (1 - d) / N + d * v)

In [ ]:
results = dict(ranks.collect())

In [ ]:
results[2582486138]

In [ ]:
results[1306760288243462147]

In [ ]:
sc.stop()